To open this notebook in Google Colab and start coding, click on the Colab icon below.

<table style="border:2px solid orange" align="left">
    <td style="border:2px solid orange">
        <a target="_blank" href="https://colab.research.google.com/github/neuefische/ds-meetups/blob/main/03_Recommender_Introduction/01_content_based.ipynb">
        <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
    </td>
</table>

# Content based recommender system

Recommender systems are algorithms which suggest information of interest to the user. If you ever used the internet, you will surely have made contact with a recommender system. Netflix, Amazon, Spotify, all make use of such functions to improve user satisfaction as well as buying behavior. Based on your previous activityThe recommender system will suggest you the next video, product, song or whatever. The most common recommender types are content based, collaborative filtering and hybrid models of those two types.

Having enough user data do build a proper collaborative model is usally an obstacle at the beginning, as there is little known about the users behaviour. This is also referred to as the cold start problem in recommender systems. In this notebook you will learn how to build a content based recommender-system. A content based model has the advantage that no user data is required and the model can be scaled to a large number of users independently of their preferences. Solving the cold start problem. For a content based recommender system the filtering is based on the item features only.
<br/><br/>
<p align="center">
    <img src="images/book_text.png" alt="drawing" width="400"/>
</p>
<br/><br/>
An example: If you have a book that has the features "aquarium" and "fish" you would get recommendations with the same features, so books about aquarium fish without any usage of ratings or user data.


Let's start with building your own content based recommender system!

Import packages:

In [152]:
#requirements
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity

Now, let's read in the data.

In [153]:
#read data
df = pd.read_csv("data/fish_1.csv")
df.reset_index(drop= True)
df.head()

,name,water_parameters,fish_group,compatibility_with_crabs,temperature,pelvic_region,behaviour,compatibility_with_fish,tank_size,breeding,...,origin,diet,vegetation_possible,compatibility_with_snails_shells,compatibility_with_dwarf_crayfish,final_size,level_of_difficulty,characteristics,compatibility_with_large_crayfish,life_expectancy
0,Cardinalfish - Tanichthys albonubes,soft to hard,Beardies,no,18-22 °C,everywhere,active,"yes, with peaceful fish",54 l (ca. 60cm),medium,...,Asia,omnivorous - omnivorous,yes,yes,yes,4-8 cm,1 - easy,dynamic group behaviour,no,3-5 years
1,Red-headed Tetra - Petitella (Hemigrammus) ble...,soft to medium hard,Tetra,NaN,25-30 °C,top,NaN,NaN,100 l (ca. 80cm),hard,...,South America,omnivorous - omnivorous,yes,yes,yes,4-8 cm,2 - normal,NaN,NaN,NaN
2,Blue Neon - Paracheirodon simulans,soft to medium hard,Tetra,NaN,25-30 °C,center,NaN,NaN,54 l (ca. 60cm),hard,...,South America,omnivorous - omnivorous,yes,yes,yes,1-4 cm,1 - easy,NaN,NaN,NaN
3,Red Neon XL - Paracheirodon axelrodi,soft to medium hard,Tetra,NaN,25-30 °C,center,NaN,NaN,54 l (ca. 60cm),hard,...,South America,omnivorous - omnivorous,yes,yes,yes,4-8 cm,1 - easy,NaN,NaN,NaN
4,Neon Tetra - Paracheirodon innesi,soft to medium hard,Tetra,NaN,25-30 °C,top,NaN,NaN,54 l (ca. 60cm),hard,...,South America,omnivorous - omnivorous,yes,yes,yes,1-4 cm,2 - normal,NaN,NaN,NaN


## Cosine similarity 

To get meaningful recommendations, it is important to measure the similarity of two items. To do so, cosine similarity is the most popular metric for content based models. Mathematically, the cosine similarity is the dot product of normalized vectors and measures the cosine of the angle between two vectors projected in a n-dimensional space (n = number of features). Output values are limited between 0 and 1, with a value of 0 indicating no similarity, whereas 1 means that both the items are identical. 

If you got a two-dimensional space and the angle of two data points is 45° the cosine would be 0.7 while it would be zero for an angle of 90°. In a data frame each feature represents a dimension. This means, every observation (row) could be plotted in a n-dimensional space based on it's feature-properties. Let's have a look at two examples:

### Example 1: 2-dimensional space 

Let's assume, we got two fish with different colors. One fish is solid blue, while the other is solid orange. Plotted in a 2-dimensional space, where one axis represents the color blue and the other the color orange, one fish would have the coordinates (1,0) and the other (0,1). A fish with both colours would be in bewtween (1,1). To calculate the cosine similarity score between two fish, we simply measure the cosine of the two coordinate vectors. 

<br/><br/>
<p align="center">
    <img src="images/fish_text.png" alt="drawing" width="300"/>
</p>
<br/><br/>
The vectors of the blue and orange fish and the solid orange fish would have a 45° angle, therefore a cosine similarity of 0.7. Just the same if we compare the solid blue and the blue and orange fish. The solid blue and the solid orange fish would have an angle of 90° and hence a cosine of 0, so no similarity at all with respect to the feature color. 

### Example 2: 4-dimensional space

In [154]:
#create example data frame
d = {'green': [1, 1, 0, 1, 1], 'blue': [1, 0, 0, 1, 1], 'yellow': [0, 0, 1, 1, 0], 'black': [1, 0, 0, 1, 1]}
df_example2 = pd.DataFrame(data = d, index= ["fish1", "fish2", "fish3", "fish4", "fish5"])
df_example2

,green,blue,yellow,black
fish1,1,1,0,1
fish2,1,0,0,0
fish3,0,0,1,0
fish4,1,1,1,1
fish5,1,1,0,1


For this example let's think of 5 different fish. Each fish has a different combination of the colors green, blue, yellow and black. Those colors are the features of the fish and likewise the dimensionality of the data. Each feature is represented by a value of 0 or 1, while 0 means the fish doesn't have this color and 1 if it has. So in this case fish1 has the colors green, blue and yellow. If we assign colors to dimensions we get the coordinates (1,1,0,1). The following equation allows us to calculate the cosine similarity for multi-dimensional data: 

$$
  CosSim(x,y) = \frac{\sum_{i}x_{i}y_{i}}{\sqrt{\sum_{i}x_{i}^2}\sqrt{\sum_{i}y_{i}^2}}
$$

If we calculate the cosine similarity on our data frame, we get:

In [155]:
#calculate cosine similarity
cosine_sim_sk = pd.DataFrame(cosine_similarity(df_example2), columns=["fish1", "fish2", "fish3", "fish4", "fish5"], index= ["fish1", "fish2", "fish3", "fish4", "fish5"])
cosine_sim_sk

,fish1,fish2,fish3,fish4,fish5
fish1,1.000000,0.57735,0.0,0.866025,1.000000
fish2,0.577350,1.00000,0.0,0.500000,0.577350
fish3,0.000000,0.00000,1.0,0.500000,0.000000
fish4,0.866025,0.50000,0.5,1.000000,0.866025
fish5,1.000000,0.57735,0.0,0.866025,1.000000


How to interpret this?

You can see, that fish1 is of course totally similar to itself, indicated by a value of 1. Then fish1 is less similar to fish 2 (only one overlap) as to fish 4 (three overlaps) and has no similarity to fish3 while it is totally similar to fish5.

## Now, let's start to build our first recommender system :)

To calculate the cosine similarity for our fishes in our original data frame, we need to transform the features into binary vectors. To do this we use the get_dummies function implemented in scikit-learn.

In [156]:
df_vectorized = pd.get_dummies(df.iloc[:, 1:])
df_vectorized.head()

,water_parameters_brackish water,water_parameters_hard water,water_parameters_medium hard to hard,water_parameters_medium hard water,water_parameters_soft to hard,water_parameters_soft to medium hard,water_parameters_soft water,fish_group_Barbs,fish_group_Beardies,fish_group_Bone Tongues,...,characteristics_growth eater,characteristics_interesting color,characteristics_interesting parental care,characteristics_snail eater,compatibility_with_large_crayfish_conditional (see description),compatibility_with_large_crayfish_no,compatibility_with_large_crayfish_yes,life_expectancy_10+ years,life_expectancy_3-5 years,life_expectancy_5-10 years
0,0,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,1,0,0,1,0
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


You can see that this increases the numbers of features from 24 to 114. 

Let's calculate the cosine similarity:

In [157]:
#calculate cosine similarity
cosine_sim_sk = pd.DataFrame(cosine_similarity(df_vectorized))

And finally the function to get recommendations.

This function takes in the name of a fish, checks the similarity scores compared to other fish, sorts the values and gives out the 10 most similar fish based on similarity score. Just type in a name of a fish in the data frame and you will get recommended the 10 most similar fish.

In [158]:
# Build a 1-dimensional array with fish names
names = df['name']
indices = pd.Series(df.index, index=df['name'])

# Function that get fish recommendations based on the cosine similarity 
def fish_recommendations(name):
    idx = indices[name]
    sim_scores = list(enumerate(cosine_sim_sk[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    fish_indices = [i[0] for i in sim_scores]
    return names.iloc[fish_indices]

Now, use this function to get recommendations :)

In [159]:
#get recommendations
fish_recommendations('Blue Neon - Paracheirodon simulans')

3                   Red Neon XL - Paracheirodon axelrodi
11              Sparkling Tetra - Hyphessobrycon amandae
26        Glowlight Tetra DNZ - Hemigrammus erythrozonus
32               Rio Red - Hyphessobrycon flammeus - DNZ
85                  Dwarf tetra - Nannostomus marginatus
128              Red Neon - Paracheirodon axelrodi - DNZ
4                      Neon Tetra - Paracheirodon innesi
5      Neon tetra "diamant" EU-NZ - Paracheirodon innesi
15                     Copper Tetra DNZ - Hasemania nana
27     Black Emperor Tetra - Nematobrycon palmeri "bl...
Name: name, dtype: object

## Text related content-based recommender

When you think of online warehouses or movie databases, many items got descriptions in text form. Content-based recommender system are often build on these descriptions as they contain useful information about the article. In the following you will find an example of how to build a text based recommender system.

First we have to create a text block, we can work with. Let's join our features to text. 

In [160]:
#set column types to string for whole data frame
df = df.astype(str)
columns = df.columns
#merge features to text block
df['text'] = df[columns].agg(' '.join, axis=1)


Let's have a look at the text.

In [161]:
#check first element
df.text[0]

'Cardinalfish - Tanichthys albonubes soft to hard Beardies no 18-22 °C everywhere active yes, with peaceful fish 54 l (ca. 60cm) medium swarm behaviour with dwarf shrimp, offspring may be eaten Asia omnivorous - omnivorous yes yes yes 4-8 cm 1 - easy dynamic group behaviour no 3-5 years'

## Text vectorization


Different tools are available to vectorize texts. Dependent on your problem, you can use NLP methods like CountVectorizer, TfidfVectorizor or words2vec. In this notebook we will use the tfidfVectorizer. Documentation can be found [here](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html). TF-IDF or Term Frequency Inverse Document Frequency is a statistical measure to determine the significance of different words in a document. Subsequently every word gets a TF-IDF value. This value is the product of the Term frequency (TF) and the inverse document frequency (IDF).</br></br>

Let's have a look at an example with 3 sentences where each sentence represents a document:

1. Recommender are cool
2. Recommender are helpful
3. Recommender use your data

In these 3 sentences the term "recommender" occurs 3 times, the therm "are" 2 times and "use" 1 time.</br></br>


Let's calculate the TF (term-frequency) for those sentences.

1. [0.33, 0.33, 0.33] 
2. [0.33, 0.33, 0.33] 
3. [0.25, 0.25, 0.25, 0.25]

In sentence 1. are 3 unique words, the frequency of each word is therefore 1/3, same in sentence 2.Sentence 4 contains of 4 unique words with a corresponding frequency of 1/4.</br></br>

Now, let's have a look at the IDF (inverse document frequency)</br></br>

The IDF is the logarithm of the ratio of the total number of documents to the number of documents having this term. 

* The word "recommender" appears in all 3 documents and has therefore a Log(3/3) = 0
* The word "are" appears in two documents with a corresponding Log(3/2) = 0.176
* The words "cool", "helpful", etc appear in only one document leading to a Log(3/1) = 0.477</br></br>

The IDF is a measure of how much information the word provides and as you can see, the IDF is higher the rarer the word is. This means rare words are more meaningful to get the context of a document.</br></br>


The TF-IDF of a word then is the product of the tf and the idf value. The higher the score, the more relevant is this word for that particular document and with regards to recommender system the more important they are to distinguish categories. 





Now that we've learned how TF-IDF works, let's vectorize our data frame:

In [162]:
#vectorize description column
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df['text'])

Since we now have a vecorized matrix of our data, we can likewise above calculate the cosine similarity on it.

In [163]:
#calculate cosine similarity
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix) 

And use the cosine similarity matrix to get our top 10 recommendations.

In [164]:
# Build a 1-dimensional array with fish names
names = df['name']
indices = pd.Series(df.index, index=df['name'])

# Function that get fish recommendations based on the cosine similarity 
def fish_recommendations(name):
    idx = indices[name]
    sim_scores = list(enumerate(cosine_sim_sk[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    fish_indices = [i[0] for i in sim_scores]
    return names.iloc[fish_indices]

In [165]:
#get recommendations
fish_recommendations('Blue Neon - Paracheirodon simulans')

3                   Red Neon XL - Paracheirodon axelrodi
11              Sparkling Tetra - Hyphessobrycon amandae
26        Glowlight Tetra DNZ - Hemigrammus erythrozonus
32               Rio Red - Hyphessobrycon flammeus - DNZ
85                  Dwarf tetra - Nannostomus marginatus
128              Red Neon - Paracheirodon axelrodi - DNZ
4                      Neon Tetra - Paracheirodon innesi
5      Neon tetra "diamant" EU-NZ - Paracheirodon innesi
15                     Copper Tetra DNZ - Hasemania nana
27     Black Emperor Tetra - Nematobrycon palmeri "bl...
Name: name, dtype: object

Inspecting the outcome, you can see that the most similar fish to the blue neon is the red neon, a close relative. If you want you can now play around with different fish names or use this notebook as a template for your own recommender :)